In [ ]:
import numpy as np
import glob
import os, re
import cv2
import random
import matplotlib.pyplot as plt
import tensorflow as tf
import datetime

In [ ]:
!pwd

In [ ]:
# 
# Data directory - CHANGE it to the correct one
# 

dir = "/Users/tomi/Astronomy/MTA_CSFK/ML_OGLE/OGLE3/test/testdata_nocolor_withtxt/"

In [ ]:
datafiles = []
datafiles = sorted(glob.glob(dir + "*.png"))
print("Number of image samples: ", len(datafiles))

In [ ]:
dataPertxts = []
dataPertxts = sorted(glob.glob(dir + "/*.txt"))
print("Number of period text samples: ", len(dataPertxts))

In [ ]:
datasize = len(datafiles)
train_setsize = 0.8
test_setsize = 0.2
# validate_setsize = 0.2
img_size = 128
batch = 32
epoch_number = 25

In [ ]:
X_Img = []
X_Ptext = []
y = []

for i in datafiles:
    img = cv2.imread(i) / 255.  
    per = np.genfromtxt(dir + str(os.path.basename(i).split(".")[0]) + ".txt")
    
    if img.shape[0] != img.shape[1]:
        img = cv2.resize(datafile,(img_size,img_size))
    label = os.path.basename(i).split("_")[0]
    X_Img.append(img)
    X_Ptext.append(per)
    y.append(label)

X_Img=np.array(X_Img)
X_Ptext=np.array(X_Ptext)
y=np.array(y)

In [ ]:
# XPtext = []
# yPtext = []

# for i in dataPertxts:
#     dataPertxt = np.genfromtxt(i)
#     label = os.path.basename(i).split("_")[0]
#     XPtext.append(dataPertxt)
#     yPtext.append(label)

# XPtext=np.array(XPtext)
# yPtext=np.array(yPtext)

In [ ]:
#print("Datasize:", datasize)
#print("Train files:", len(train_set))
#print("Test files:", len(test_set))
#print("Control files:", len(control_set))


In [ ]:
import sklearn.preprocessing
encoder = sklearn.preprocessing.OneHotEncoder(sparse=False)

encoder.fit(y.reshape(-1,1))
y_enc = encoder.transform(y.reshape(-1,1))

# encoder.fit(yPtext.reshape(-1,1))
# yPtext_enc = encoder.transform(yPtext.reshape(-1,1))

In [ ]:
import sklearn.model_selection
X_Img_train, X_Img_test, y_Img_train, y_Img_test = sklearn.model_selection.train_test_split(X_Img, y_enc, 
                                                                            shuffle=True, 
                                                                            stratify=y, 
                                                                            train_size = train_setsize,
                                                                            test_size=test_setsize, 
                                                                            random_state=42)

(n_sampl, x_dim, y_dim, lc_channels) = X_Img_train.shape


In [ ]:
# X_Ptext_train, X_Ptext_test, y_Ptext_train, y_Ptext_test = sklearn.model_selection.train_test_split(X_Ptext, y_enc, 
#                                                                             shuffle=True, 
#                                                                             stratify=yPtext, 
#                                                                             train_size = train_setsize,
#                                                                             test_size=test_setsize, 
#                                                                             random_state=42)

X_Ptext_train, X_Ptext_test = sklearn.model_selection.train_test_split(X_Ptext, 
                                                                            shuffle=True, 
                                                                            stratify=y, 
                                                                            train_size = train_setsize,
                                                                            test_size=test_setsize, 
                                                                            random_state=42)

# XPtext_train[0]
(nPtext_sampl) = X_Ptext_train.shape

In [ ]:
print(len(X_train))

In [ ]:
# X_Img_train.shape

In [ ]:
# y_Img_train.shape

In [ ]:
# X_Ptext_train.shape

In [ ]:
# 
# Test network version 0.2 - multi layer DNN
# Multi-layer network using the functional API
# 

from tensorflow.keras import Model
from tensorflow.keras.layers import Input,Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Flatten,Dense,Dropout,concatenate

lc_inputs = Input(shape=(x_dim, y_dim, lc_channels))
lcx = Conv2D(16, 8, activation='relu')(lc_inputs)
lcx = MaxPooling2D( pool_size=(2,2) )(lcx)

lcx = Conv2D(32, 4, activation='relu')(lcx)
lcx = MaxPooling2D( pool_size=(2,2) )(lcx)

lcx = Conv2D(64, 2, activation='relu')(lcx)
lcx = MaxPooling2D( pool_size=(2,2) )(lcx)

lcx = Conv2D(128, 1, activation='relu')(lcx)
lcx = MaxPooling2D( pool_size=(2,2) )(lcx)

lcx = Conv2D(256, 1, activation='relu')(lcx)
lcx = MaxPooling2D( pool_size=(2,2) )(lcx)

lcx = Flatten()(lcx)

Ptext_inputs = Input(shape=(1,))
ptxty = Flatten()(Ptext_inputs)

model = concatenate([lcx, ptxty])

model = Dense(256, activation='relu')(model)
model = Dense(256, activation='relu')(model)
model = Dense(64, activation='relu')(model)

model = Dense(5, activation='softmax')(model)

model = Model(inputs=[lc_inputs,Ptext_inputs], outputs = model)


In [ ]:
# 
# Multi GPU model - use it ONLY when you train with GPUs
# 

from tensorflow.keras.utils import multi_gpu_model
model = multi_gpu_model(model, gpus=4)

In [ ]:
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import TensorBoard

log_dir = dir + '/logs/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback = TensorBoard(log_dir,histogram_freq=1)

opt = SGD(lr=0.00001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [ ]:
model.fit([X_Img_train, X_Ptext_train],
          y_Img_train,
          validation_data=([X_Img_train, X_Ptext_train],y_Img_train,)
          batch_size=batch, 
          verbose=1, 
          epochs=epoch_number,
          callbacks=[tensorboard_callback])

# model.fit([X_Img_train, X_Ptext_train],
#           y_Img_train,
#           batch_size=batch, 
#           verbose=1, 
#           epochs=epoch_number)


In [ ]:
#################
# Saving the model (architecture, weights, training configuration, state of the optimizer)
#################

# model.save('OGLE3_colored.h5')
model.save('OGLE3_non_colored_with_text.h5')

# !!!!!
# Eddig kell a kódot lefuttatni!
# !!!!!

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print ('Loss, accuracy:', test_loss, test_accuracy)


In [ ]:
train_loss, train_accuracy = model.evaluate(X_train, y_train)
print ('Loss, accuracy:', test_loss, test_accuracy)

In [ ]:
import keras.utils
from keras.utils.vis_utils import plot_model
# import tensorflow.keras.utils.plot_model as modelplot
# tensorflow.keras.utils.
keras.Model.summary(model)
# plot_model(model, to_file='model.png')


In [ ]:
import tensorflow.keras.utils as utils

utils.plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)

In [ ]:
a=encoder.inverse_transform(y_test)
b=model.predict(X_test)
#encoder.transform(np.argmax(b,axis=1))
#print(np.argmax(b[4]),a[4])
#plt.imshow(X_test[4])
#print(b[4], y_test[4])
b[0]

In [ ]:
from keras.preprocessing import image

img = "/Users/tomi/Astronomy/MTA_CSFK/ML_OGLE/ACep/ACEP_LCs/acep_038.png"
# img = "/Users/tomi/Astronomy/MTA_CSFK/ML_OGLE/DSct/DSct_LCs/dsct_2735.png"
img = "/Users/tomi/Astronomy/MTA_CSFK/ML_OGLE/ECL/ECL_LCs_2/01015.png"
# img = "/Users/tomi/Astronomy/MTA_CSFK/ML_OGLE/RRLyr/RRLyr_LCs/rrlyr_01020.png"
# img = "/Users/tomi/Astronomy/MTA_CSFK/ML_OGLE/T2Cep/T2CEP_LCs/t2cep_157.jpg"

# img = cv2.imread(img)
# img = cv2.resize(img,(img_size,img_size))

# img = np.expand_dims(img, axis=0)

# model.predict(img)
img = image.load_img(img, target_size=(img_size, img_size))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
model.predict(img)
model.predict_classes(img)



In [ ]:
# from keras.utils.vis_utils import plot_model
# from tensorflow.keras.utils
import pydot

# keras.utils.plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
keras.utils.plot_model(model, show_shapes=True, show_layer_names=True)

In [ ]:
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
# Summaries for histogram

k = tf.placeholder(tf.float32)
# Make a normal distribution, with a shifting mean
mean_moving_normal = tf.random_normal(shape=[100], mean=(5*k), stddev=1)
# Record that distribution into a histogram summary
tf.summary.histogram("normal/moving_mean", mean_moving_normal)
